In [28]:
import copy
import http.client
import json
import tqdm
import time

def print_json(data: dict) -> None:
    print(json.dumps(data, indent=4 ))

class LolDataFetcher:

    def __init__(self):
        self.conn = http.client.HTTPSConnection("cdn.merakianalytics.com")

    def get_patches(self) -> list[str]:
        print(f"Requesting patches from cdn.merakianalytics.com")
        t = time.time()
        self.conn.request("GET", "/riot/lol/resources/patches.json")
        response = self.conn.getresponse()
        data = response.read()
        data = json.loads(data.decode("utf-8"))["patches"]

        print(f"Received {len(data)} patches in {time.time() - t:.2f} seconds")
        patch_names = [patch["name"] for patch in data]

        return patch_names

    def get_latest_items(self) -> dict:
        print(f"Requesting latest items from cdn.merakianalytics.com")
        t = time.time()
        endpoint = "/riot/lol/resources/latest/en-US/items.json"
        self.conn.request("GET", endpoint)
        response = self.conn.getresponse()
        data = response.read()
        data = json.loads(data.decode("utf-8"))
        print(f"Received {len(data.keys())} items in {time.time() - t:.2f} seconds")
        return data

    def get_latest_champions(self) -> dict:
        print(f"Requesting latest champions from cdn.merakianalytics.com")
        t = time.time()
        endpoint = "/riot/lol/resources/latest/en-US/champions.json"
        self.conn.request("GET", endpoint)
        response = self.conn.getresponse()
        data = response.read()
        data = json.loads(data.decode("utf-8"))
        print(f"Received {len(data.keys())} champions in {time.time() - t:.2f} seconds")
        return data


class DataPreProcessor:
    def __init__(self, verbose: bool = True) -> None:
        self.verbose = verbose

    def get_items_list(self, items_data: dict) -> list[dict]:
        item_id_names = {}
        for k, v in items_data.items():
            item_id_names[k] = v["name"]

        items = []
        if self.verbose:
            print(f"Preprocessing  {len(items_data.keys())} items")
            pbar = tqdm.tqdm(total=len(items_data), desc="Preprocessing items")
        for item in items_data.values():
            items.append(self.preprocess_item(item, item_id_names))
            if self.verbose:
                pbar.set_description(f"Preprocessing {item['name']}")
                pbar.update(1)
        return items

    def get_champions_list(self, champions_data: dict) -> list[dict]:
        champions = []

        if self.verbose:
            print(f"Preprocessing {len(champions_data.keys())} champions")
            pbar = tqdm.tqdm(total=len(champions_data), desc="Preprocessing champions")
        for champion in champions_data.values():
            champions.append(self.preprocess_champion(champion))
            if self.verbose:
                pbar.set_description(f"Preprocessing {champion['name']}")
                pbar.update(1)
        return champions

    def preprocess_champion(self, champion: dict) -> dict:
        new_champion = copy.deepcopy(champion)

        new_champion.pop("id", None)
        new_champion.pop("icon", None)
        new_champion.pop("skins", None)

        new_champion["stats"] = self.preprocess_stats(new_champion["stats"])
        for ability_k in new_champion["abilities"].keys():
            new_champion["abilities"][ability_k] = [
                self.preprocess_ability(new_champion["abilities"][ability_k][i])
                for i in range(len(new_champion["abilities"][ability_k]))
            ]

        return new_champion

    def preprocess_ability(self, ability: dict) -> dict:

        new_ability = copy.deepcopy(ability)
        new_ability.pop("icon", None)
        
        def process_modifier_recurs( input_dict : dict) -> None:
            for k , v in input_dict.items():
                if k == "modifiers":
                    input_dict[k] = [self.preprocess_modifier(modifier) for modifier in v]
                elif isinstance(v, dict):
                    process_modifier_recurs(v)
                elif isinstance(v, list):
                    for item in v:
                        if isinstance(item, dict):
                            process_modifier_recurs(item)
        process_modifier_recurs(new_ability)
        return new_ability

    def preprocess_modifier(self, modifier: dict) -> dict:
        new_modifer = {}
        new_modifer["valuePerLevel"] = []
        for i in range(len(modifier["values"])):
            new_modifer["valuePerLevel"].append(
                f"{modifier['values'][i] :.3f}  {modifier['units'][i]}"
            )
        return new_modifer

    def preprocess_item(self, item: dict, item_id_names: dict) -> dict:
        new_item = copy.deepcopy(item)

        # remove unecessary fields
        new_item.pop("id", None)
        new_item.pop("noEffects", None)
        new_item.pop("removed", None)
        new_item.pop("requiredAlly", None)
        new_item.pop("icon", None)
        new_item.pop("specialRecipe", None)
        new_item.pop("iconOverlay", None)

        # remove stats with 0 values
        new_item["stats"] = self.preprocess_stats(new_item["stats"])

        # convert item ids to item names

        builds_from_names = [
            item_id_names[str(id)]
            for id in new_item["buildsFrom"]
            if str(id) in item_id_names
        ]
        builds_into_names = [
            item_id_names[str(id)]
            for id in new_item["buildsInto"]
            if str(id) in item_id_names
        ]

        new_item["buildsFrom"] = builds_from_names
        new_item["buildsInto"] = builds_into_names

        #
        for passive in new_item["passives"]:
            if "stats" in passive:
                passive["stats"] = self.preprocess_stats(passive["stats"])

        for active in new_item["active"]:
            if "stats" in active:
                active["stats"] = self.preprocess_stats(active["stats"])

        return new_item

    def preprocess_stats(self, stats: dict) -> dict:
        stats_new = copy.deepcopy(stats)
        for k, v in list(stats_new.items()):
            for stat_key, stat_value in list(v.items()):
                if stat_value == 0:
                    stats_new[k].pop(stat_key)

            if len(stats_new[k]) == 0:
                stats_new.pop(k)
        return stats_new


fetcher = LolDataFetcher()
preProcessor = DataPreProcessor()
patches = fetcher.get_patches()
# last_patch = patches[0]


# champions = fetcher.get_champions_brief(last_patch)
# champions
last_patch = patches[-1]
print(last_patch)
items_raw = fetcher.get_latest_items()
champions = fetcher.get_latest_champions()
items_processed = preProcessor.get_items_list(items_raw)
champions_processed = preProcessor.get_champions_list(champions)
# print(items_processed[200])
print_json(champions_processed[0]["abilities"])

Requesting patches from cdn.merakianalytics.com
Received 267 patches in 0.31 seconds
14.13
Requesting latest items from cdn.merakianalytics.com


Preprocessing champions:   0%|          | 0/167 [26:02<?, ?it/s]


Received 265 items in 2.93 seconds
Requesting latest champions from cdn.merakianalytics.com
Received 167 champions in 12.14 seconds
Preprocessing  265 items


Preprocessing Guardian's Dirk: 100%|██████████| 265/265 [00:01<00:00, 176.11it/s]


Preprocessing 167 champions


Preprocessing Zyra: 100%|██████████| 167/167 [00:01<00:00, 131.71it/s]

{
    "P": [
        {
            "name": "Deathbringer Stance",
            "effects": [
                {
                    "description": "Innate: Periodically, Aatrox empowers his next basic attack to gain 50 bonus range and deal bonus magic damage equal to 4% \u2212 12% (based on level) of the target's maximum health, capped at 100 against monsters. Aatrox heals for 80% of the post-mitigation bonus damage dealt, reduced to 25% against minions.",
                    "leveling": []
                },
                {
                    "description": "Whenever Aatrox hits at least one enemy champion or large monster with a basic attack on-hit or an ability, the cooldown of Deathbringer Stance is reduced by 2 seconds, modified to 4 if he hits with the Sweetspot of The Darkin Blade.",
                    "leveling": []
                }
            ],
            "cost": null,
            "cooldown": {
                "modifiers": [
                    {
                        "

In [2]:
import requests

url = "https://ddragon.leagueoflegends.com/cdn/14.13.1/data/en_US/champion/Aatrox.json"

data = requests.get(url)
data = data.json()
data = data["data"]["Aatrox"]
data

{'id': 'Aatrox',
 'key': '266',
 'name': 'Aatrox',
 'title': 'the Darkin Blade',
 'image': {'full': 'Aatrox.png',
  'sprite': 'champion0.png',
  'group': 'champion',
  'x': 0,
  'y': 0,
  'w': 48,
  'h': 48},
 'skins': [{'id': '266000', 'num': 0, 'name': 'default', 'chromas': False},
  {'id': '266001', 'num': 1, 'name': 'Justicar Aatrox', 'chromas': False},
  {'id': '266002', 'num': 2, 'name': 'Mecha Aatrox', 'chromas': True},
  {'id': '266003', 'num': 3, 'name': 'Sea Hunter Aatrox', 'chromas': False},
  {'id': '266007', 'num': 7, 'name': 'Blood Moon Aatrox', 'chromas': False},
  {'id': '266008',
   'num': 8,
   'name': 'Prestige Blood Moon Aatrox',
   'chromas': False},
  {'id': '266009', 'num': 9, 'name': 'Victorious Aatrox', 'chromas': True},
  {'id': '266011', 'num': 11, 'name': 'Odyssey Aatrox', 'chromas': True},
  {'id': '266020',
   'num': 20,
   'name': 'Prestige Blood Moon Aatrox (2022)',
   'chromas': False},
  {'id': '266021', 'num': 21, 'name': 'Lunar Eclipse Aatrox', 'chro